In [1]:
# Import packages to be used
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import re

In [2]:
# IMDB Top 250 Movies webpage (starting point) 
link = 'https://www.imdb.com/chart/top/?ref_=nv_mv_250' # IMDB top 250 movies link
path = "C:\\Users\mhard\\Desktop\\chromedriver.exe" # my chromedriver lives on my desktop and is called "chromedriver"
s=Service(path)
browser = webdriver.Chrome(service=s)
browser.get(link)
time.sleep(2)

# Parse to a soup
page_source = browser.page_source
soup = BeautifulSoup(page_source, 'lxml')
url_content = soup.find_all('tr')

# Create list of URL's to be used to scrape webpage for each individual movie
url_List = []
for url in url_content:
    url = url.find('a', href=True)    
    if url != None:
        url_List.append('https://imdb.com'+str(url['href']))
    else:
        url_List.append('N/A')
browser.close()       
url_List = url_List[1:]                    

In [3]:
# Setting up lists to hold data to be retrieved
theMost = len(url_List) # Loop stopping point
movieTitleList = [] # Movie Title List 
releaseYearList = [] # Release Year List
imdbUserRatingList = [] # IMDB User Rating List
imdbUserRatingCountList = [] # IMDB User Rating Count List
imdbUserPopularityList = [] # IMDB User Popularity List
potentialGenres = ['Action', 'Adult', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',       # List 
                   'Documentary', 'Drama', 'Family', 'Fantasy', 'Film', 'Noir', 'Game Show',          # of
                   'History', 'Horror', 'Musical', 'Music', 'Mystery', 'News', 'Reality-TV',          # Potential
                   'Romance', 'Sci-Fi', 'Short', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western']  # Genres
movieGenreList = [] # Movie Genre List
userReviewCountList = [] # IMDB User Review Count List
criticReviewCountList = [] # Critic Review Count List
metaScoreList = [] # Metascore List
movieDescriptionList = [] # Movie Description List
directorList = [] # Movie Director List
actorList = [] # Movie Actor List
budgetList = [] # Movie Budget List
x = 1 # Counter to catch movies without budget listed 

# Begin loop
for i in range (0, theMost):
    link = str(url_List[i]) # universal portion of rotten tomatoes movie review url
    path = "C:\\Users\mhard\\Desktop\\chromedriver.exe" # my chromedriver lives on my desktop and is called "chromedriver"
    s=Service(path)
    browser = webdriver.Chrome(service=s)
    browser.get(link)
    time.sleep(2)
    page_source = browser.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    
    #MOVIE TITLE DATA
    titleContent = soup.find('h1')
    
    if titleContent == None:
        movieTitleList.append('N/A')
    else:
        movieTitleList.append(titleContent.text)
    
    #RELEASE YEAR DATA
    yearContent = soup.find('span', class_='sc-8c396aa2-2 itZqyK')
    
    if yearContent == None:
        releaseYearList.append('N/A')
    else:
        releaseYearList.append(yearContent.text)
    
    #IMDB RATING DATA
    imdbUserRatingContent = soup.find('span', class_='sc-7ab21ed2-1 jGRxWM')
    
    if imdbUserRatingContent == None:
        imdbUserRatingList.append('N/A')
    else:
        imdbUserRatingList.append(imdbUserRatingContent.text)
    
    #IMDB RATING (COUNT) DATA
    imdbUserRatingCountContent = soup.find('div', class_='sc-7ab21ed2-3 dPVcnq')
    
    if imdbUserRatingCountContent == None:
        imdbUserRatingCountList.append('N/A')
    else:
        imdbUserRatingCountList.append(imdbUserRatingCountContent.text)
    
    #IMDB POPULARITY DATA
    imdbUserPopularityContent = soup.find('div', class_='sc-edc76a2-1 gopMqI')
    
    if imdbUserPopularityContent == None:
        imdbUserPopularityList.append('N/A')
    else:
        imdbUserPopularityContent = imdbUserPopularityContent.text
        imdbUserPopularityList.append(int(imdbUserPopularityContent.replace("$", "").replace(",", "")))
    
    #MOVIE GENRE DATA
    movieGenreGroup = []
    movieGenreContent = soup.findAll('span', class_='ipc-chip__text')
    
    if movieGenreContent == None:
        for movieGenre in movieGenreContent:
            movieGenreGroup.append('NA')
        movieGenreList.append(movieGenreGroup)
    else:
        for movieGenre in movieGenreContent:
            if movieGenre.text in potentialGenres:
                movieGenreGroup.append(movieGenre.text)
        movieGenreList.append(movieGenreGroup)
    
    #USER & CRITIC REVIEW COUNT DATA
    reviewCountContent = soup.findAll('span', class_='score')
    userReviewCountContent = reviewCountContent[0]
    criticReviewCountContent = reviewCountContent[1]
    userReviewCountList.append(userReviewCountContent.text)
    criticReviewCountList.append(criticReviewCountContent.text)
    
    #METASCORE DATA
    metaScoreContent = soup.find('span', class_='score-meta')    
    
    if metaScoreContent == None:
        metaScoreList.append('N/A')
    else:
        metaScoreList.append(metaScoreContent.text)
    
    #MOVIE DESCRIPTION DATA
    movieDescriptionContent = soup.find('span', class_='sc-16ede01-1 kgphFu')
    
    if movieDescriptionContent == None:
        movieDescriptionList.append('N/A')
    else:
        movieDescriptionList.append(movieDescriptionContent.text)

    #DIRECTOR DATA
    directorContent = soup.find('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')
    
    if directorContent == None:
        directorList.append('N/A')
    else:
        directorList.append(directorContent.text)
    
    #ACTOR DATA
    actorGroup = []
    actorContent = soup.findAll('a', class_='sc-bfec09a1-1 gfeYgX')
    actorGroup.append(actorContent[0].text)
    actorGroup.append(actorContent[1].text)
    actorGroup.append(actorContent[2].text)
    actorList.append(actorGroup)
    
    #BOX OFFICE DATA
    boxOfficeContent = soup.findAll('label', class_='ipc-metadata-list-item__list-content-item')
    
    for boxOffice in boxOfficeContent:
        if boxOffice.text.endswith('(estimated)'):
            budgetList.append(boxOffice.text)
            break
    
    if len(budgetList) < (x):
        budgetList.append('N/A')
    
    x+=1    

    print(i+1)
    browser.close()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250


In [4]:
# List to store cleaned version of 'imdbUserRatingCountList'
cleanimdbUserRatingCountList = []
for imdbUserRatingCount in imdbUserRatingCountList:
    if str(imdbUserRatingCount).endswith('M'):
        imdbUserRatingCount = str(imdbUserRatingCount[:-1])
        imdbUserRatingCount = float(imdbUserRatingCount) * 1000000
        imdbUserRatingCount = int(imdbUserRatingCount)
        cleanimdbUserRatingCountList.append(imdbUserRatingCount)
    
    elif str(imdbUserRatingCount).endswith('K'):
        imdbUserRatingCount = str(imdbUserRatingCount[:-1])
        imdbUserRatingCount = float(imdbUserRatingCount) * 1000
        imdbUserRatingCount = int(imdbUserRatingCount)
        cleanimdbUserRatingCountList.append(imdbUserRatingCount)

# List to store cleaned version of 'userReviewCountList'
cleanuserReviewCountList = []
for userReviewCount in userReviewCountList:
    if str(userReviewCount).endswith('M'):                   # Handle millions (M)
        userReviewCount = str(userReviewCount[:-1])
        userReviewCount = float(userReviewCount) * 1000000
        userReviewCount = int(userReviewCount)
        cleanuserReviewCountList.append(userReviewCount)
    
    elif str(userReviewCount).endswith('K'):                 # Handle thousands (K)
        userReviewCount = str(userReviewCount[:-1])
        userReviewCount = float(userReviewCount) * 1000
        userReviewCount = int(userReviewCount)
        cleanuserReviewCountList.append(userReviewCount)
    else:                                                    # Handle < thousands
        userReviewCount = str(userReviewCount)
        userReviewCount = float(userReviewCount)
        userReviewCount = int(userReviewCount)
        cleanuserReviewCountList.append(userReviewCount)

# Lists to store cleaned version of 'budgetList' & extract listed currency for budget 
currencyList = []        
newBudgetList = []
for budget in budgetList:
    if budget.__contains__('$'):               # Handle USD
        currencyList.append('USD')
        budget = re.sub("[^0-9]", "", budget)
        newBudgetList.append(budget)
    elif budget.__contains__('¥'):             # Handle ¥
        currencyList.append('YEN')
        budget = re.sub("[^0-9]", "", budget)
        newBudgetList.append(budget)
    elif budget.__contains__('FRF'):           # Handle FRF
        currencyList.append('FRANC')
        budget = re.sub("[^0-9]", "", budget)
        newBudgetList.append(budget)
    elif budget.__contains__('€'):             # Handle €
        currencyList.append('EURO')
        budget = re.sub("[^0-9]", "", budget)
        newBudgetList.append(budget)
    elif budget.__contains__('DEM'):           # Handle DEM
        currencyList.append('DEUTSCHMARK')
        budget = re.sub("[^0-9]", "", budget)
        newBudgetList.append(budget)
    elif budget.__contains__('₹'):             # Handle ₹
        currencyList.append('RUPEE')
        budget = re.sub("[^0-9]", "", budget)
        newBudgetList.append(budget)
    elif budget.__contains__('£'):             # Handle £
        currencyList.append('GBP')
        budget = re.sub("[^0-9]", "", budget)
        newBudgetList.append(budget)        
    elif budget.__contains__('MVR'):           # Handle MVR
        currencyList.append('RUFIYAA')
        budget = re.sub("[^0-9]", "", budget)
        newBudgetList.append(budget)
    elif budget.__contains__('₩'):             # Handle ₩
        currencyList.append('WON')
        budget = re.sub("[^0-9]", "", budget)
        newBudgetList.append(budget)          
    elif budget.__contains__('N/A'):           # Handle movies without budget listed
        currencyList.append('N/A')
        budget = re.sub("[^0-9]", "", budget)
        newBudgetList.append('N/A')

In [5]:
# Create dataframe from lists of data that have been scraped
df_Top250Movies=pd.DataFrame(zip(movieTitleList, releaseYearList, imdbUserRatingList, 
                          cleanimdbUserRatingCountList, imdbUserPopularityList, 
                          movieGenreList, cleanuserReviewCountList, criticReviewCountList, 
                          metaScoreList, movieDescriptionList, directorList, actorList, 
                          budgetList, newBudgetList, currencyList), 
                      columns=['movieTitle', 'releaseYear', 'imdbUserRating', 
                               'imdbUserRatingCount', 'imdbUserPopularity', 
                               'movieGenre(s)', 'userReviewCount', 'criticReviewCount', 
                               'metaScore', 'movieDescription', 'movieDirector', 'headlineActors', 
                               'movieBudget', 'newMovieBudget', 'budgetCurrency'])

# Preview dataframe
df_Top250Movies

,movieTitle,releaseYear,imdbUserRating,imdbUserRatingCount,imdbUserPopularity,movieGenre(s),userReviewCount,criticReviewCount,metaScore,movieDescription,movieDirector,headlineActors,movieBudget,newMovieBudget,budgetCurrency
0,The Shawshank Redemption,1994,9.3,2700000,107,[Drama],10500,170,81,Two imprisoned men bond over a number of years...,Frank Darabont,"[Tim Robbins, Morgan Freeman, Bob Gunton]","$25,000,000 (estimated)",25000000,USD
1,The Godfather,1972,9.2,1900000,85,"[Crime, Drama]",5200,195,100,The aging patriarch of an organized crime dyna...,Francis Ford Coppola,"[Marlon Brando, Al Pacino, James Caan]","$6,000,000 (estimated)",6000000,USD
2,The Dark Knight,2008,9.0,2600000,154,"[Action, Crime, Drama]",8300,442,84,When the menace known as the Joker wreaks havo...,Christopher Nolan,"[Christian Bale, Heath Ledger, Aaron Eckhart]","$185,000,000 (estimated)",185000000,USD
3,The Godfather Part II,1974,9.0,1300000,310,"[Crime, Drama]",1300,134,90,The early life and career of Vito Corleone in ...,Francis Ford Coppola,"[Al Pacino, Robert De Niro, Robert Duvall]","$13,000,000 (estimated)",13000000,USD
4,12 Angry Men,1957,9.0,790000,416,"[Crime, Drama]",2000,154,96,The jury in a New York City murder trial is fr...,Sidney Lumet,"[Henry Fonda, Lee J. Cobb, Martin Balsam]","$350,000 (estimated)",350000,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,The Iron Giant,1999,8.1,200000,2184,"[Animation, Action, Adventure]",630,142,85,A young boy befriends a giant robot from outer...,Brad Bird,"[Eli Marienthal, Harry Connick Jr., Jennifer A...","$70,000,000 (estimated)",70000000,USD
246,The Help,2011,8.1,461000,829,[Drama],647,298,62,An aspiring author during the civil rights mov...,Tate Taylor,"[Viola Davis, Emma Stone, Octavia Spencer]","$25,000,000 (estimated)",25000000,USD
247,Aladdin,1992,8.0,421000,1625,"[Animation, Adventure, Comedy]",386,152,86,A kindhearted street urchin and a power-hungry...,Ron Clements,"[Scott Weinger, Robin Williams, Linda Larkin]","$28,000,000 (estimated)",28000000,USD
248,Everything Everywhere All at Once,2022,8.1,255000,32,"[Action, Adventure, Comedy]",2200,346,81,An aging Chinese immigrant is swept up in an i...,Dan Kwan,"[Michelle Yeoh, Stephanie Hsu, Jamie Lee Curtis]","$25,000,000 (estimated)",25000000,USD


In [6]:
# Save dataframe as .CSV file so that data is available to be used without running code again
df_Top250Movies.to_csv("IMDB_Top250Movies.csv")